In [49]:
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd
import numpy as np
import html5lib
import requests
from openpyxl import Workbook
from datetime import datetime
import webbrowser
import os

In [71]:
path = '../Test_GBDC_Investment.xlsx'
xls = pd.ExcelFile(path)
all_sheets = pd.read_excel(path, sheet_name=None)
dataframes = {}
# Loop through each sheet and create a DataFrame in the dictionary
for sheet_name, sheet_df in all_sheets.items():
    dataframes[sheet_name.replace(' ', '_')] = sheet_df
    print(f"DataFrame name: {sheet_name} : {sheet_df.shape}")

DataFrame name: March 31 2013 : (424, 31)
DataFrame name: June 30 2013 : (432, 31)
DataFrame name: September 30 2013 : (486, 41)
DataFrame name: December 31 2013 : (443, 31)
DataFrame name: March 31 2014 : (467, 31)
DataFrame name: June 30 2014 : (482, 27)
DataFrame name: September 30 2014 : (527, 33)
DataFrame name: December 31 2014 : (506, 31)
DataFrame name: March 31 2015 : (524, 31)
DataFrame name: June 30 2015 : (571, 31)
DataFrame name: September 30 2015 : (584, 33)
DataFrame name: December 31 2015 : (505, 27)
DataFrame name: March 31 2016 : (589, 27)
DataFrame name: June 30 2016 : (564, 27)
DataFrame name: September 30 2016 : (700, 23)
DataFrame name: December 31 2016 : (646, 23)
DataFrame name: March 31 2017 : (677, 23)
DataFrame name: June 30 2017 : (699, 23)
DataFrame name: December 31 2017 : (734, 41)
DataFrame name: March 31 2018 : (766, 25)
DataFrame name: June 30 2018 : (817, 25)
DataFrame name: September 30 2018 : (883, 24)
DataFrame name: December 31 2018 : (916, 41)
Da

In [ ]:
def process_table_fun(soi_table_df, process_tables_shapes):
    print(1, 'shape:', soi_table_df.shape)
    soi_table_df = soi_table_df.replace(r'^\s*\$\s*$', np.nan, regex=True)
    soi_table_df = soi_table_df.replace(r'\n', '', regex=True)
    print(2, 'shape:', soi_table_df.shape)
    # fewer non-null values, effectively dropping rows with more than 100 non-null values.
    # for index, row in soi_table_df.iterrows():
    #     if row.count() > 100:
    #         soi_table_df = soi_table_df.drop(index)

    pattern = r'Total\s+Investments'
    # Use the apply function to check if the pattern is in any column for each row
    matching_rows = soi_table_df.apply(
        lambda row: row.str.contains(pattern, flags=re.IGNORECASE, regex=True).any(), axis=1)
    # Find the index of the first row that matches the pattern
    # Slice the DataFrame to keep only the rows up to and including the first matching row
    if soi_table_df[matching_rows].index[0] < 20:
        soi_table_df = soi_table_df.loc[:soi_table_df[matching_rows].index[1]].reset_index(
            drop=True)
    else:
        soi_table_df = soi_table_df.loc[:soi_table_df[matching_rows].index[0]].reset_index(
            drop=True)

    print(0, 'shape:', soi_table_df.shape)
    soi_table_df = soi_table_df.dropna(how='all', axis=1).reset_index(
        drop=True)

    print(3, 'shape:', soi_table_df.shape)
    soi_table_df = soi_table_df.dropna(how='all', axis=0).reset_index(
        drop=True)

    print(4, 'shape:', soi_table_df.shape)
    soi_table_df.dropna().reset_index(
        drop=True)
    print(5, 'shape:', soi_table_df.shape)

    pattern = r'Net asset value per common share|How We Addressed the Matter in Our Audit'
    matching_rows = soi_table_df.apply(
        lambda row: row.str.contains(pattern, flags=re.IGNORECASE, regex=True).any(), axis=1)

    # Check if the pattern exists in the DataFrame
    if matching_rows.any():
        # Extract rows from the first occurrence onwards
        soi_table_df = soi_table_df.iloc[matching_rows.idxmax(
        )+1:].reset_index(drop=True)

    # Reset the index after making modifications
    soi_table_df.reset_index(drop=True, inplace=True)

    # pattern = r'(?i)investment\s*type?|investment'
    # pattern = 'Investments'
    # matching_rows = soi_table_df.apply(
    #     lambda row: row.str.contains(pattern, flags=re.IGNORECASE, regex=True).any(), axis=1)

    # if matching_rows is not None:
    #     soi_table_df = soi_table_df.loc[soi_table_df[matching_rows].index[0]:].reset_index(
    #         drop=True)

    # second_row_values = soi_table_df.iloc[1]
    # for col_idx, value in enumerate(second_row_values):
    #     if not pd.isna(value) and not pd.isna(soi_table_df.columns[col_idx]):
    #         col_name = soi_table_df.columns[col_idx]

    #         soi_table_df.rename(columns={col_name: new_col_name}, inplace=True)
    #         soi_table_df.iloc[2, col_idx] = np.nan
    # print(soi_table_df.columns)

    # soi_table_df = soi_table_df.rename(
    #     columns=soi_table_df.iloc[0]).drop(soi_table_df.index[0])

    print(6, 'shape:', soi_table_df.shape)
    soi_table_df = soi_table_df.replace('—', 0)
    soi_table_df = soi_table_df.replace('-', np.nan)
    print(7, 'shape:', soi_table_df.shape)

    soi_table_df = soi_table_df.applymap(
        lambda x: x.strip() if isinstance(x, str) else x)

    process_tables_shapes.append(soi_table_df.shape)

    return soi_table_df

In [ ]:
path = '/Users/fuadhassan/Desktop/BDC_RA/GBDC/process_tables_GBDC_Investment.xlsx'
writer = pd.ExcelWriter(path, engine='openpyxl')
process_tables = {}
process_tables_shape = []
for dataframe in dataframes:
    print(dataframe)
    process_tables[dataframe] = process_table_fun(
        dataframes[dataframe], process_tables_shape)
    print('\n')
    process_tables[dataframe].to_excel(
        writer, sheet_name=dataframe.replace(',', ''), index=False)
    writer.book .save(path)
writer.close()

In [ ]:
process_tables['March_31_2013'].to_csv('test.csv')

In [44]:
soi_table_df = process_tables['March_31_2013']

In [45]:
soi_table_df

,Investments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Leisure, Amusement, Motion Pictures,",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Entertainment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"Extreme Fitness, Inc.",Senior loan,NaN,NaN,NaN,10.00,%,03/2013,NaN,220,NaN,220,NaN,NaN,%,220,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416,Vision Source L.P.,Common stock,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN
417,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5389,NaN,1.2,NaN,5667,NaN
418,Total equity investments United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23540,NaN,5.2,%,26023,NaN
419,Total United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,780136,NaN,157.6,%,787520,NaN


In [31]:

pattern = r'(?i)investment\s*type?|investment'
matching_rows = soi_table_df.apply(
    lambda row: row.str.contains(pattern, flags=re.IGNORECASE, regex=True).any(), axis=1)
soi_table_df = soi_table_df.loc[soi_table_df[matching_rows].index[0]:].reset_index(
    drop=True)
soi_table_df

,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Percentage,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,Investment,Spread Above,NaN,Interest,NaN,Maturity,Principal,NaN,NaN,of,NaN,Fair,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cost,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Investments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Debt investments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418,Vision Source L.P.,Common stock,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN
419,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5389,NaN,1.2,NaN,5667,NaN
420,Total equity investments United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23540,NaN,5.2,%,26023,NaN
421,Total United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,780136,NaN,157.6,%,787520,NaN


In [36]:
df = soi_table_df

# Define the regular expression pattern to match the variations
pattern = r'(?i)investment\s*type?|investment'

# Check if any rows match the pattern
matching_rows = soi_table_df[soi_table_df.astype(str).apply(lambda x: x.str.contains(
    pattern, flags=re.IGNORECASE, regex=True))]
matching_rows

,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Percentage,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,Investment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Investments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Debt investments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
419,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
420,Total equity investments United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
421,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
matching_rows

In [37]:
pattern = r'(?i)investment\s*type?|investment'

# Create a boolean mask for rows where the pattern doesn't match
no_match_mask = ~soi_table_df.astype(str).apply(
    lambda x: x.str.contains(pattern, flags=re.IGNORECASE, regex=True)).any(axis=1)
# Filter rows based on the mask
no_match_mask


0      False
1       True
2      False
3       True
4      False
       ...  
418     True
419     True
420    False
421     True
422    False
Length: 423, dtype: bool

In [46]:
pattern = r'(?i)investment\s*type?|investment'
# pattern = r'Spread Above'
matching_rows = soi_table_df.apply(
    lambda row: row.str.contains(pattern, flags=re.IGNORECASE, regex=True).any(), axis=1)
soi_table_df = soi_table_df[matching_rows]
soi_table_df

,Investments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,Debt investments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,"Road Infrastructure Investment, LLC (4)",Senior loan,NaN,L + 5.00%,NaN,NaN,(5),03/2017,NaN,NaN,NaN,(37,NaN,NaN,NaN,(1,NaN
81,"Road Infrastructure Investment, LLC*",Senior loan,NaN,L + 5.00%,NaN,6.25,%,03/2018,NaN,4116,NaN,4065,NaN,0.8,NaN,4147,NaN
84,NaN,Investment,Spread Above,NaN,Interest,NaN,Maturity,Principal,NaN,NaN,of Total,NaN,Fair,NaN,NaN,NaN,NaN
159,NaN,Investment,Spread Above,NaN,Interest,NaN,Maturity,Principal,NaN,NaN,of Total,NaN,Fair,NaN,NaN,NaN,NaN
237,NaN,Investment,Spread Above,NaN,Interest,NaN,Maturity,Principal,NaN,NaN,of Total,NaN,Fair,NaN,NaN,NaN,NaN
320,Total debt investments United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,768823,NaN,756596,NaN,152.4,%,761497,NaN
323,NaN,Investment,Spread Above,NaN,Interest,NaN,Maturity,Amount/Shares/,NaN,NaN,of Total,NaN,Fair,NaN,NaN,NaN,NaN
325,Equity investments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
407,NaN,Investment,Spread Above,NaN,Interest,NaN,Maturity,Amount/Shares/,NaN,NaN,of Total,NaN,Fair,NaN,NaN,NaN,NaN


In [48]:
pattern = r'(?:Spread\s*Above|cost|Percentage)'

matching_rows = soi_table_df.apply(
    lambda row: row.str.contains(pattern, flags=re.IGNORECASE, regex=True).any(), axis=1)
soi_table_df = soi_table_df[matching_rows]
soi_table_df

,Investments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84,NaN,Investment,Spread Above,NaN,Interest,NaN,Maturity,Principal,NaN,NaN,of Total,NaN,Fair,NaN,NaN,NaN,NaN
159,NaN,Investment,Spread Above,NaN,Interest,NaN,Maturity,Principal,NaN,NaN,of Total,NaN,Fair,NaN,NaN,NaN,NaN
237,NaN,Investment,Spread Above,NaN,Interest,NaN,Maturity,Principal,NaN,NaN,of Total,NaN,Fair,NaN,NaN,NaN,NaN
323,NaN,Investment,Spread Above,NaN,Interest,NaN,Maturity,Amount/Shares/,NaN,NaN,of Total,NaN,Fair,NaN,NaN,NaN,NaN
407,NaN,Investment,Spread Above,NaN,Interest,NaN,Maturity,Amount/Shares/,NaN,NaN,of Total,NaN,Fair,NaN,NaN,NaN,NaN


In [170]:
def process_table_fun(soi_table_df, process_tables_shapes):
    print(1, 'shape:', soi_table_df.shape)
    soi_table_df = soi_table_df.replace(r'^\s*\$\s*$', np.nan, regex=True)
    soi_table_df = soi_table_df.replace(r'\n', '', regex=True)
    print(2, 'shape:', soi_table_df.shape)
    print(6, 'shape:', soi_table_df.shape)
    soi_table_df = soi_table_df.replace('—', 0)
    soi_table_df = soi_table_df.replace('-', 0)
    print(7, 'shape:', soi_table_df.shape)
    soi_table_df.reset_index(drop=True, inplace=True)

    soi_table_df = soi_table_df.applymap(
        lambda x: x.strip() if isinstance(x, str) else x)

    pattern = r'Total\s+Investments'
    # Use the apply function to check if the pattern is in any column for each row
    matching_rows = soi_table_df.apply(
        lambda row: row.str.contains(pattern, flags=re.IGNORECASE, regex=True).any(), axis=1)
    # Find the index of the first row that matches the pattern
    # Slice the DataFrame to keep only the rows up to and including the first matching row
    if soi_table_df[matching_rows].index[0] < 20:
        soi_table_df = soi_table_df.loc[:soi_table_df[matching_rows].index[1]].reset_index(
            drop=True)
    else:
        soi_table_df = soi_table_df.loc[:soi_table_df[matching_rows].index[0]].reset_index(
            drop=True)

# removest end extra
    pattern = r'Net asset value per common share|How We Addressed the Matter in Our Audit'
    matching_rows = soi_table_df.apply(
        lambda row: row.str.contains(pattern, flags=re.IGNORECASE, regex=True).any(), axis=1)

    # Check if the pattern exists in the DataFrame
    if matching_rows.any():
        # Extract rows from the first occurrence onwards
        soi_table_df = soi_table_df.iloc[matching_rows.idxmax(
        )+1:].reset_index(drop=True)

    # removing all col name
    pattern = r'(?:Spread\s*Above|cost|Percentage|Above)'
    matching_rows = soi_table_df.apply(
        lambda row: row.str.contains(pattern, flags=re.IGNORECASE, regex=True).any(), axis=1)
    soi_table_df = soi_table_df[~matching_rows]

    print(0, 'shape:', soi_table_df.shape)
    soi_table_df = soi_table_df.dropna(how='all', axis=1).reset_index(
        drop=True)
    print(3, 'shape:', soi_table_df.shape)
    soi_table_df = soi_table_df.dropna(how='all', axis=0).reset_index(
        drop=True)
    print(4, 'shape:', soi_table_df.shape)
    soi_table_df.dropna().reset_index(
        drop=True)
    print(5, 'shape:', soi_table_df.shape)


# drops the sub total
    soi_table_df = soi_table_df.dropna(subset=[soi_table_df.columns[0]])

    num_cols = soi_table_df.shape[1]
    data_col_mapper = dict(zip(soi_table_df.columns.to_list(), [
        i for i in range(0, num_cols)]))
    soi_table_df = soi_table_df.rename(columns=data_col_mapper)

    # for index, row in soi_table_df.iterrows():
    #     for column in soi_table_df.columns:
    #         if pd.isna(row[column]):
    #             # Find the next column in the same row
    #             next_column = soi_table_df.columns.get_loc(column) + 1
    #             if next_column < len(soi_table_df.columns):
    #                 next_column_name = soi_table_df.columns[next_column]
    #                 # Replace NaN value with the value from the next column
    #                 soi_table_df.at[index, column] = row[next_column_name]
    #                 # Set the next column to NaN
    #                 soi_table_df.at[index, next_column_name] = np.nan

    for index, row in soi_table_df.iterrows():
        for column in soi_table_df.columns:
            if pd.isna(row[column]):
                while True:
                    # Find the next column in the same row
                    next_column = soi_table_df.columns.get_loc(column) + 1
                    if next_column < len(soi_table_df.columns):
                        next_column_name = soi_table_df.columns[next_column]
                        # Replace NaN value with the value from the next column
                        soi_table_df.at[index, column] = row[next_column_name]
                        # Set the next column to NaN
                        soi_table_df.at[index, next_column_name] = np.nan
                        column = next_column_name
                    else:
                        # No more columns to replace, break out of the loop
                        break

    soi_table_df = soi_table_df.dropna(axis=1, thresh=10)
    soi_table_df = soi_table_df.dropna(how='all', axis=1).reset_index(
        drop=True)

    process_tables_shapes.append(soi_table_df.shape)
    print(soi_table_df.info())

    return soi_table_df


path = '/Users/fuadhassan/Desktop/BDC_RA/GBDC/process_tables_GBDC_Investment.xlsx'
writer = pd.ExcelWriter(path, engine='openpyxl')
process_tables = {}
process_tables_shape = []
for dataframe in dataframes:
    print(dataframe)
    process_tables[dataframe] = process_table_fun(
        dataframes[dataframe], process_tables_shape)
    print('\n')
    process_tables[dataframe].to_excel(
        writer, sheet_name=dataframe.replace(',', ''), index=False)
    writer.book .save(path)
writer.close()

March_31_2013
1 shape: (424, 31)
2 shape: (424, 31)
6 shape: (424, 31)
7 shape: (424, 31)
0 shape: (404, 31)
3 shape: (404, 11)
4 shape: (404, 11)
5 shape: (404, 11)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 370 entries, 0 to 369
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       370 non-null    object
 1   1       310 non-null    object
 2   2       249 non-null    object
 3   3       251 non-null    object
 4   4       315 non-null    object
 5   5       315 non-null    object
 6   6       315 non-null    object
 7   7       314 non-null    object
 8   8       240 non-null    object
 9   9       170 non-null    object
dtypes: object(10)
memory usage: 29.0+ KB
None


June_30_2013
1 shape: (432, 31)
2 shape: (432, 31)
6 shape: (432, 31)
7 shape: (432, 31)
0 shape: (408, 31)
3 shape: (408, 21)
4 shape: (408, 21)
5 shape: (408, 21)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 370 entries, 0 to 369
Data column

In [112]:
pattern = r'(?:Spread\s*Above|cost|Percentage|Above)'
matching_rows = soi_table_df.apply(
    lambda row: row.str.contains(pattern, flags=re.IGNORECASE, regex=True).any(), axis=1)
soi_table_df = soi_table_df[~matching_rows]
soi_table_df

,1,2,4,5,6,8,9,10,11,13,...,17,18,19,20,21,22,23,24,25,26
0,Investments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Non-controlled/non-affiliate company investments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Debt investments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Aerospace and Defense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"PPW Aero Buyer, Inc.+",One stop,SF +,7.00 %,(m),11.90 %,NaN,NaN,NaN,02/2029,...,NaN,NaN,NaN,24187,NaN,NaN,1.0,%,NaN,24888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1458,MMan Acquisition Co.+(23),One stop,NaN,NaN,NaN,12.00 %,PIK,NaN,NaN,08/2023,...,NaN,275,NaN,NaN,0,NaN,275,NaN,NaN,NaN
1459,MMan Acquisition Co.+(23),One stop,NaN,NaN,NaN,12.00 %,PIK,NaN,NaN,08/2023,...,NaN,849,NaN,NaN,0,NaN,849,NaN,NaN,NaN
1460,Total controlled affiliate company debt invest...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,23261,NaN,NaN,0.5,%,NaN,13579,NaN
1461,Total controlled affiliate company investments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,23261,NaN,NaN,0.5,%,NaN,13579,NaN,NaN,NaN


In [113]:
soi_table_df = soi_table_df.dropna(subset=[soi_table_df.columns[0]])
soi_table_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1392 entries, 0 to 1462
Data columns (total 22 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   1       1392 non-null   object
 1   2       1294 non-null   object
 2   4       1026 non-null   object
 3   5       1026 non-null   object
 4   6       851 non-null    object
 5   8       1063 non-null   object
 6   9       152 non-null    object
 7   10      116 non-null    object
 8   11      116 non-null    object
 9   13      1045 non-null   object
 10  15      1268 non-null   object
 11  16      32 non-null     object
 12  17      4 non-null      object
 13  18      1255 non-null   object
 14  19      16 non-null     object
 15  20      80 non-null     object
 16  21      1214 non-null   object
 17  22      58 non-null     object
 18  23      1252 non-null   object
 19  24      34 non-null     object
 20  25      12 non-null     object
 21  26      32 non-null     object
dtypes: object(22)
memory usage: 2

In [150]:
soi_table_df = process_tables['December_31_2013']

for index, row in soi_table_df.iterrows():
    for column in soi_table_df.columns:
        if pd.isna(row[column]):

            # Find the next column in the same row
            next_column = soi_table_df.columns.get_loc(column) + 1
            if next_column < len(soi_table_df.columns):
                next_column_name = soi_table_df.columns[next_column]
                # Replace NaN value with the value from the next column
                soi_table_df.at[index, column] = row[next_column_name]
                # Set the next column to NaN
                soi_table_df.at[index, next_column_name] = np.nan


soi_table_df.to_csv('test.csv')

In [166]:
soi_table_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 382 entries, 0 to 381
Data columns (total 19 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       382 non-null    object
 1   1       333 non-null    object
 2   2       333 non-null    object
 3   3       333 non-null    object
 4   4       333 non-null    object
 5   5       333 non-null    object
 6   6       333 non-null    object
 7   7       329 non-null    object
 8   8       264 non-null    object
 9   9       260 non-null    object
 10  10      184 non-null    object
 11  11      0 non-null      object
 12  12      0 non-null      object
 13  13      0 non-null      object
 14  14      0 non-null      object
 15  15      0 non-null      object
 16  16      0 non-null      object
 17  17      0 non-null      object
 18  18      0 non-null      object
dtypes: object(19)
memory usage: 56.8+ KB


In [156]:
soi_table_df = process_tables['December_31_2013']

for index, row in soi_table_df.iterrows():
    columns_to_fill = []
    for column in soi_table_df.columns:
        if pd.notna(row[column]):
            if columns_to_fill:
                for col_to_fill in columns_to_fill:
                    soi_table_df.at[index, col_to_fill] = row[column]
                columns_to_fill = []
        else:
            columns_to_fill.append(column)

soi_table_df.to_csv('test_2.csv')
soi_table_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,Investments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Debt investments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Aerospace and Defense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"ILC Dover, LP",Senior loan,P + 5.00,%,8.25,%,07/2017,21,15,0,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377,Telecommunications,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
378,"ITC Global, Inc.",Preferred stock,17,17,17,311,0.1,311,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
379,Total equity investments United States,33626,33626,33626,33626,6.5,42605,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
380,Total United States,1166589,1166589,1166589,1166589,178.8,1179919,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [155]:
soi_table_df = process_tables['December_31_2013']

for index, row in soi_table_df.iterrows():
    for column in soi_table_df.columns:
        if pd.isna(row[column]):
            while True:
                # Find the next column in the same row
                next_column = soi_table_df.columns.get_loc(column) + 1
                if next_column < len(soi_table_df.columns):
                    next_column_name = soi_table_df.columns[next_column]
                    # Replace NaN value with the value from the next column
                    soi_table_df.at[index, column] = row[next_column_name]
                    # Set the next column to NaN
                    soi_table_df.at[index, next_column_name] = np.nan
                    column = next_column_name
                else:
                    # No more columns to replace, break out of the loop
                    break

soi_table_df.to_csv('test.csv')
soi_table_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,Investments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Debt investments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Aerospace and Defense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"ILC Dover, LP",Senior loan,P + 5.00,%,8.25,%,07/2017,21,15,0,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377,Telecommunications,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
378,"ITC Global, Inc.",Preferred stock,NaN,NaN,17,311,0.1,311,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
379,Total equity investments United States,NaN,NaN,NaN,33626,6.5,42605,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
380,Total United States,NaN,NaN,NaN,1166589,178.8,1179919,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [159]:
dataframes['December_31_2013'].to_csv('Test_3.csv')

In [171]:
process_tables_shape

[(370, 10),
 (370, 14),
 (386, 10),
 (382, 14),
 (403, 12),
 (416, 12),
 (436, 10),
 (434, 12),
 (449, 10),
 (477, 10),
 (487, 10),
 (430, 15),
 (515, 11),
 (479, 10),
 (592, 13),
 (587, 15),
 (614, 16),
 (640, 15),
 (671, 17),
 (707, 17),
 (745, 16),
 (786, 16),
 (849, 13),
 (883, 15),
 (925, 15),
 (1021, 15),
 (1073, 16),
 (1116, 14),
 (1115, 14),
 (1147, 18),
 (1185, 17),
 (1237, 14),
 (1318, 16),
 (1429, 14),
 (1443, 14),
 (1541, 17),
 (1634, 17),
 (1627, 16),
 (1630, 16),
 (1470, 16),
 (1392, 16)]